In [1]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools
import seaborn as sns

In [2]:
df = pd.read_csv('../data/dataHasilPreprocessing/hasilPreprocessing1.csv')
df['Sentimen'] = df['Sentimen'].map({'Positif': 1, 'Negatif': 0})
df

Ulasan  Sentimen
0                                                 ulas         0
1                                                  NaN         0
2                        bahan aja warna navy nya beda         0
3    kasih catat order warna kuning navy baca kirim...         0
4                                                kecil         0
..                                                 ...       ...
990  barang selamat terima kasih bahan celana nya s...         0
991                                              kecil         0
992    pesan cuna barang rusak coba konfirmasi tanggap         0
993                         karet pinggang nya kencang         0
994                           aja mahal sih hrg segitu         0

[995 rows x 2 columns]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ulasan    983 non-null    object
 1   Sentimen  995 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


In [4]:
df = df.drop_duplicates(subset=['Ulasan'])

In [5]:
df = df.dropna()

In [6]:
tb_counts = df.Sentimen.value_counts()
tb_counts

Sentimen
1    812
0    135
Name: count, dtype: int64

# IndoBert

In [7]:
model_name = 'indobenchmark/indobert-base-p1'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
reviews = df['Ulasan'].tolist()
labels = df['Sentimen'].tolist()  

In [9]:
# Tokenize data
max_length = 128  # Panjang maksimum input ke model
input_ids = []
attention_masks = []

for review in reviews:
    encoded_dict = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=max_length,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
labels = tf.convert_to_tensor(labels)

In [11]:
# Split data into train and test sets
train_indices, test_indices = train_test_split(range(len(input_ids)), test_size=0.2, random_state=42)

# Konversi indeks numpy ke tensor TensorFlow
train_indices = tf.convert_to_tensor(train_indices, dtype=tf.int32)
test_indices = tf.convert_to_tensor(test_indices, dtype=tf.int32)

# Gunakan indeks tensor untuk mengindeks data pelatihan dan pengujian
train_input_ids = tf.gather(input_ids, train_indices)
train_attention_masks = tf.gather(attention_masks, train_indices)
train_labels = tf.gather(labels, train_indices)

test_input_ids = tf.gather(input_ids, test_indices)
test_attention_masks = tf.gather(attention_masks, test_indices)
test_labels = tf.gather(labels, test_indices)

In [12]:
# Define training configuration
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [13]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [14]:
# Train the model
batch_size = 16
epochs = 15

history = model.fit(
    [train_input_ids, train_attention_masks],
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([test_input_ids, test_attention_masks], test_labels)
)

Epoch 1/15
48/48 [==============================] - 26s 261ms/step - loss: 0.4562 - accuracy: 0.8362 - val_loss: 0.3950 - val_accuracy: 0.8789
Epoch 2/15
48/48 [==============================] - 11s 220ms/step - loss: 0.2351 - accuracy: 0.9128 - val_loss: 0.2526 - val_accuracy: 0.9000
Epoch 3/15
48/48 [==============================] - 11s 221ms/step - loss: 0.1312 - accuracy: 0.9604 - val_loss: 0.3560 - val_accuracy: 0.8737
Epoch 4/15
48/48 [==============================] - 11s 224ms/step - loss: 0.0855 - accuracy: 0.9709 - val_loss: 0.5317 - val_accuracy: 0.8368
Epoch 5/15
48/48 [==============================] - 11s 222ms/step - loss: 0.0496 - accuracy: 0.9908 - val_loss: 0.5245 - val_accuracy: 0.8737
Epoch 6/15
48/48 [==============================] - 11s 224ms/step - loss: 0.0225 - accuracy: 0.9960 - val_loss: 0.5903 - val_accuracy: 0.8737
Epoch 7/15
48/48 [==============================] - 11s 224ms/step - loss: 0.0327 - accuracy: 0.9894 - val_loss: 0.5467 - val_accuracy: 0.8684

In [15]:
# Evaluate the model
model.evaluate([test_input_ids, test_attention_masks], test_labels)

6/6 [==============================] - 1s 136ms/step - loss: 0.7936 - accuracy: 0.8684


[0.7936198711395264, 0.8684210777282715]

In [16]:
# Make predictions
test_predictions = model.predict([test_input_ids, test_attention_masks])
predicted_labels = tf.argmax(test_predictions.logits, axis=1)

# Show classification report
print(classification_report(test_labels, predicted_labels))

6/6 [==============================] - 3s 132ms/step
              precision    recall  f1-score   support

           0       0.46      0.48      0.47        23
           1       0.93      0.92      0.92       167

    accuracy                           0.87       190
   macro avg       0.69      0.70      0.70       190
weighted avg       0.87      0.87      0.87       190

